In [22]:
import numpy as np
import string
import json
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Bidirectional
from keras.layers.core import SpatialDropout1D
from sklearn.model_selection import StratifiedKFold
from keras.datasets import imdb
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

from gensim import corpora

# def parsePhrases(stopWords, engStemmer, phrases):
#     print "parse the phrases with stopwords and stemmer"
#     processedPhrases = []
#     for phrase in phrases:
#         tokens = word_tokenize(phrase)
#         parsedWords = []
#         for t in tokens:
#             if t not in stopWords:
#                 parsedWords.append(engStemmer.stem(t))
#         processedPhrases.append(parsedWords)
#     return processedPhrases
postProcessedTrainPhrases = []
postProcessedTestPhrases = []

def preprocessData():
    print("Loading and preprocessing data...")
    # load training and testing data
    with open('labeled_document2.json') as json_data:
        allTrainData = json.load(json_data)


    trainPhrases, testPhrases, trainLabel,testLabel = train_test_split(allTrainData['Comment'], allTrainData['CommentLabel'], test_size=0.2, random_state=42)
#     print(testPhrases[0:100])
    punctuation = list(string.punctuation)
    stopWords = stopwords.words('english') + punctuation 

    engStemmer = SnowballStemmer('english')
    #postProcessedTrainPhrases = []
#     for phrase in trainPhrases:
#         uni_doc = unicode(phrase, errors='replace')
#         tokens = word_tokenize(uni_doc)
#         filtered = [word for word in tokens if word not in stop_words]
#         try:
#             stemmed = [stemmer.stem(word) for word in filtered]
#         except UnicodeDecodeError:
#             print(word)
#         postProcessedTrainPhrases.append(parsedWords)

#     for phrase in testPhrases:
#         uni_doc = unicode(phrase, errors='replace')
#         tokens = word_tokenize(uni_doc)
#         filtered = [word for word in tokens if word not in stop_words]
#         try:
#             stemmed = [stemmer.stem(word) for word in filtered]
#         except UnicodeDecodeError:
#             print(word)
#         postProcessedTestPhrases.append(parsedWords)
    for phrase in trainPhrases:
        if not isinstance(phrase, str):
            continue
        tokens = word_tokenize(phrase)
        parsedWords = []
        for t in tokens:
            if t not in stopWords:
                parsedWords.append(engStemmer.stem(t))
        postProcessedTrainPhrases.append(parsedWords)

    for phrase in testPhrases:
        if not isinstance(phrase, str):
            continue
        tokens = word_tokenize(phrase)
        parsedWords = []
        for t in tokens:
            if t not in stopWords:
                parsedWords.append(engStemmer.stem(t))
        postProcessedTestPhrases.append(parsedWords)
    return (trainLabel,testLabel)


def convertPhrasesToIDs(phrases):
    print ("converting the phrases to id to be processed")
    wordIDs = []
    wordIDLens = []
    for phrase in phrases:
        ids = []
        for word in phrase:
            ids.append(toIDMap.token2id[word])
        wordIDs.append(ids)
        wordIDLens.append(len(ids))
    return ( wordIDs, wordIDLens )

def findSequenceLen(wordListLen):
    print( "calculate the norm sequence length")
    wordLenMean = np.mean(wordListLen)
    wordLenStd = np.std(wordListLen)
    return np.round(wordLenMean + 3 * wordLenStd).astype(int)



In [23]:
(trainSenti, testSenti) = preprocessData()
# process training data and testing data






Loading and preprocessing data...


In [25]:
toIDMap = corpora.Dictionary(np.concatenate((postProcessedTrainPhrases, postProcessedTestPhrases), axis=0))
allPhraseSize = len(toIDMap.keys())

(trainWordIDs, trainWordIDLens) = convertPhrasesToIDs(postProcessedTrainPhrases)
(testWordIDs, testWordIDLens) = convertPhrasesToIDs(postProcessedTestPhrases)

sequenceLen = findSequenceLen(trainWordIDLens + testWordIDLens)

print( "pad sequence")
trainingData = sequence.pad_sequences(np.array(trainWordIDs), maxlen=sequenceLen)
testingData = sequence.pad_sequences(np.array(testWordIDs), maxlen=sequenceLen)

print ("categorize the labels")
#print len(np.unique(trainSenti))
trainingDataLabel = np_utils.to_categorical(trainSenti, len(np.unique(trainSenti)))
testingDataLabel = np_utils.to_categorical(testSenti, len(np.unique(testSenti)))
# cvscores = []
# batchSize = 15606
# end = 156060
# for i in range(0, 10):
#     print("Fold:%d" % i)
#     tests = trainingData[i * batchSize:(i + 1) * batchSize]
#     tests_label = trainingDataLabel[i * batchSize:(i + 1) * batchSize]
#     if (i > 0):
#         trains = np.concatenate((trainingData[0:(i * batchSize)], trainingData[(i + 1) * batchSize:end]), axis=0)
#         trains_label = np.concatenate((trainingDataLabel[0:(i * batchSize)], trainingDataLabel[(i + 1) * batchSize:end]), axis=0)
#     else:
#         trains = trainingData[(i + 1) * batchSize:end]
#         trains_label = trainingDataLabel[(i + 1) * batchSize:end]

model = Sequential()
model.add(Embedding(allPhraseSize, 128))
model.add(SpatialDropout1D(0.1))
model.add(Bidirectional(LSTM(128)))
#model.add(Bidirectional(LSTM(128)))
#model.add(Flatten())
model.add(Dense(len(np.unique(trainSenti))))
model.add(Activation('softmax'))

# model = Sequential()
# model.add(Embedding(allPhraseSize, 128, dropout=0.2))
# model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))
# model.add(Dense(num_labels))
# model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(trainingData,trainingDataLabel , epochs=3, batch_size=256, verbose=1)
# evaluate the model
scores = model.evaluate(testingData, testingDataLabel, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))

converting the phrases to id to be processed
converting the phrases to id to be processed
calculate the norm sequence length
pad sequence
categorize the labels
Epoch 1/3
3394/3394 [==============================] - 11s 3ms/step - loss: 0.5299 - acc: 0.8079
Epoch 2/3
3394/3394 [==============================] - 9s 3ms/step - loss: 0.4493 - acc: 0.8385
Epoch 3/3
3394/3394 [==============================] - 8s 2ms/step - loss: 0.4183 - acc: 0.8385
acc: 82.10%
